In [64]:
#importing all the requied modules

from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import glob
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from glob import glob
import seaborn as sns
from PIL import Image
np.random.seed(123)
from sklearn.preprocessing import label_binarize
from sklearn.metrics import confusion_matrix
import itertools

import keras
from keras.utils.np_utils import to_categorical # used for converting labels to one-hot-encoding
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras import backend as K
import itertools
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding

from keras.optimizers import adam_v2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split

In [65]:
#import all the images from the folder
import glob
full_set=glob.glob('../input/skin-images/ham_data_surnamesABCD/ham_data_surnamesABCD/*.*')

In [66]:
#appending every image 

full_set=[]
path=("../input/skin-images/ham_data_surnamesABCD/ham_data_surnamesABCD/*.*")
for i in glob.glob(path):
    s1=cv2.imread(i)
    full_set.append(s1)

In [67]:
#loading the csv file

nRowsRead = None
full_csv = pd.read_csv('../input/skin-csv/filename_to_category_map_surnamesABCD.csv', delimiter=',', nrows = nRowsRead)
full_csv.dataframeName = 'filename_to_category_map_surnamesABCD.csv'
n_Row, n_Col = full_csv.shape
print(f'There are {n_Row} rows and {n_Col} columns')

In [68]:
full_csv_rep = pd.read_csv('../input/skin-csv/filename_to_category_map_surnamesABCD.csv')

In [69]:
print(full_csv)

In [70]:
len_csv = full_csv["image_id"]
print(len(len_csv))
map_images = {}
for i in len_csv:
    path = "../input/skin-images/ham_data_surnamesABCD/ham_data_surnamesABCD/" + i + ".jpg"
    if i not in map_images.keys():
        map_images[i] = path

In [71]:
full_csv['path'] = full_csv['image_id'].map(lambda x : map_images[x])

In [72]:
full_csv['image'] = full_csv['path'].map(lambda x: np.asarray(Image.open(x).resize((100,75))))

In [73]:
full_csv = full_csv.dropna()

In [74]:
full_csv.head()

In [75]:
X=full_csv["image"]
Y=full_csv["cell_type"]

In [91]:
class_name = ['Benign keratosis-like lesions ','Dermatofibroma','Melanoma','Vascular lesions','Basal cell carcinoma','Melanocytic nevi','Actinic keratoses']


plt.figure(figsize=(20,20))
for i in range(0,16) :
    plt.subplot(10,3,i+1)
    plt.axis('off')
    plt.imshow(full_set[i], cmap="gray_r")
    

In [95]:
X=full_csv["image"]
Y=full_csv["cell_type"]

In [96]:
print(X)

In [97]:
x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(X, Y, test_size=0.30,random_state=6969)

In [98]:
x_train = np.asarray(x_train_o.tolist())
x_test = np.asarray(x_test_o.tolist())

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

In [99]:
y_train = to_categorical(np.asarray(y_train_o.factorize()[0]))
y_test = to_categorical(np.asarray(y_test_o.factorize()[0]))

In [100]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size = 0.30, random_state = 69)

In [131]:
input_shape = (75, 100, 3)
num_classes = 7

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',padding = 'Same',input_shape=input_shape))
model.add(Conv2D(32,kernel_size=(3, 3), activation='relu',padding = 'Same',))
model.add(MaxPool2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(Conv2D(64, (3, 3), activation='relu',padding = 'Same'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.40))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()


In [132]:
model.summary()

In [134]:
optimizer = adam_v2.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_learning_rate=0.00001)

In [136]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

In [ ]:
epochs = 50

batch_size = 10
history = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_validate,y_validate),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size
                              , callbacks=[learning_rate_reduction])
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

